## Data Cleaning II

This is a continuation of the Data Cleaning I project. This notebook takes care of other data cleaning techniques.

#### Text & Categorical Data Problems
Categorical and text data can often be some of the messiest parts of a dataset due to their unstructured nature. In this notebook, I’ll try to : 
- fix whitespace and capitalization inconsistencies in category labels 
- collapse multiple categories into one 
- and reformat strings for consistency

The dataset to be used here is one that contains answers to survey questions on the San Francisco Airport from airline customers. The DataFrame contains flight metadata such as the airline, the destination, waiting times as well as answers to key questions regarding cleanliness, safety, and satisfaction. Another DataFrame named categories was created, containing all correct possible values for the survey columns.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Loading airlines data 
airlines = pd.read_csv('../Datasets/airlines_final.csv')
airlines.head(3)

,Unnamed: 0,id,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction
0,0,1351,Tuesday,UNITED INTL,KANSAI,Asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied
1,1,373,Friday,ALASKA,SAN JOSE DEL CABO,Canada/Mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied
2,2,2820,Thursday,DELTA,LOS ANGELES,West US,Hub,Gates 40-48,2018-12-31,70.0,Average,Somewhat safe,Neutral


In [3]:
# Creating the categories dataframe

#Creating the dictionary that will be used to create the dataframe
data = {'cleanliness' : ['Clean', 'Average', 'Somewhat clean', 'Somewhat dirty', 'Dirty'],
        'safety': ['Neutral', 'Very safe', 'Somewhat safe', 'Very unsafe', 'Somewhat unsafe'],
        'satisfaction': ['Very satisfied', 'neutral', 'Somewhat satisfied', 'Somewhat unsatisfied', 'Very unsatisfied']
       }

#Creating the dataframe
categories = pd.DataFrame(data)
# Print categories DataFrame
print(categories)

      cleanliness           safety          satisfaction
0           Clean          Neutral        Very satisfied
1         Average        Very safe               neutral
2  Somewhat clean    Somewhat safe    Somewhat satisfied
3  Somewhat dirty      Very unsafe  Somewhat unsatisfied
4           Dirty  Somewhat unsafe      Very unsatisfied


In the airlines dataframe above, the entries in the columns `cleanliness`, `safety`, and `satisfaction` should match the those in the categories dataframe. 

Take a look at the output. Out of the cleanliness, safety and satisfaction columns, which one has an inconsistent category and what is it?

In [4]:
airlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2477 entries, 0 to 2476
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2477 non-null   int64  
 1   id             2477 non-null   int64  
 2   day            2477 non-null   object 
 3   airline        2477 non-null   object 
 4   destination    2477 non-null   object 
 5   dest_region    2477 non-null   object 
 6   dest_size      2477 non-null   object 
 7   boarding_area  2477 non-null   object 
 8   dept_time      2477 non-null   object 
 9   wait_min       2477 non-null   float64
 10  cleanliness    2477 non-null   object 
 11  safety         2477 non-null   object 
 12  satisfaction   2477 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 251.7+ KB


In [5]:
#Checking the inconsistencies across the two dataframes categories and airlines 
inconsistent_categories = set(airlines['cleanliness']).difference(categories['cleanliness'])
print(inconsistent_categories)

inconsistent_categories1 = set(airlines['safety']).difference(categories['safety'])
print(inconsistent_categories1)

inconsistent_categories2 = set(airlines['satisfaction']).difference(categories['satisfaction'])
print(inconsistent_categories2)

set()
set()
{'Neutral', 'Somewhat satsified'}


Two inconsistent entries found in the satisfaction category. ('Neutral', Somewhat satsified')

In [6]:
# Find the safety category in airlines not in categories
cat_satisfaction = set(airlines['satisfaction']).difference(categories['satisfaction'])
print(cat_satisfaction)

# Find rows with that category
cat_satisfaction_rows = airlines['satisfaction'].isin(cat_satisfaction)
print(airlines[cat_satisfaction_rows].head())
len(cat_satisfaction_rows)

{'Neutral', 'Somewhat satsified'}
   Unnamed: 0    id        day     airline  destination    dest_region  \
2           2  2820   Thursday       DELTA  LOS ANGELES        West US   
3           3  1157    Tuesday   SOUTHWEST  LOS ANGELES        West US   
4           4  2992  Wednesday    AMERICAN        MIAMI        East US   
6           6  2578   Saturday     JETBLUE   LONG BEACH        West US   
7           8  2592   Saturday  AEROMEXICO  MEXICO CITY  Canada/Mexico   

  dest_size boarding_area   dept_time  wait_min     cleanliness  \
2       Hub   Gates 40-48  2018-12-31      70.0         Average   
3       Hub   Gates 20-39  2018-12-31     190.0           Clean   
4       Hub   Gates 50-59  2018-12-31     559.0  Somewhat clean   
6     Small    Gates 1-12  2018-12-31      63.0           Clean   
7       Hub    Gates 1-12  2018-12-31     215.0  Somewhat clean   

          safety        satisfaction  
2  Somewhat safe             Neutral  
3      Very safe  Somewhat satsified  
4

2477

#### Examining Dest Region (dest_region) and Dest size (dest_size) columns

In [7]:
#Looking at all the different destination regions in the data 
airlines['dest_region'].unique()

array(['Asia', 'Canada/Mexico', 'West US', 'East US', 'Midwest US',
       'EAST US', 'Middle East', 'Europe', 'eur', 'Central/South America',
       'Australia/New Zealand', 'middle east'], dtype=object)

From the results of the above code, it can be seen that East US appears twice (one in all caps and the other 'East US) Europe has been coded as eur and Europe all at the same time. East US and EAST US represent the same place coded differently. These issues will be addressed in the next cell. 

In [8]:
#Converting the strings in dest_region to lower
airlines['dest_region'] = airlines['dest_region'].str.lower()
print(airlines['dest_region'].unique())
print('')
#Number of different dest_regions
print(airlines['dest_region'].nunique(), 'different regions')

['asia' 'canada/mexico' 'west us' 'east us' 'midwest us' 'middle east'
 'europe' 'eur' 'central/south america' 'australia/new zealand']

10 different regions


In [9]:
#Changing eur to europe
airlines['dest_region'] = airlines['dest_region'].replace({'eur':'europe'})
print(airlines['dest_region'].unique())
print('')
#Number of different dest_regions
print(airlines['dest_region'].nunique(), 'different regions after cleaning')

['asia' 'canada/mexico' 'west us' 'east us' 'midwest us' 'middle east'
 'europe' 'central/south america' 'australia/new zealand']

9 different regions after cleaning


In [10]:
# Examining dest size column

#Looking at the different destination sizes in the dest size column
airlines['dest_size'].unique()

array(['Hub', 'Small', '    Hub', 'Medium', 'Large', 'Hub     ',
       '    Small', 'Medium     ', '    Medium', 'Small     ',
       '    Large', 'Large     '], dtype=object)

There is a lot of needless white space around most of the text. eg 'Hub', 'Medium'.

In [11]:
#Removing the white spaces
airlines['dest_size'] = airlines['dest_size'].str.strip(' ')
airlines['dest_size'].unique()

array(['Hub', 'Small', 'Medium', 'Large'], dtype=object)

#### The Waiting Time Column (wait_min)
 

In [12]:
#Min and Max waiting times
print('The least waiting time is', round(airlines['wait_min'].min(),2), 'minutes.')
print('The maximum waiting time is', round(airlines['wait_min'].max(), 2), 'minutes')

The least waiting time is 15.0 minutes.
The maximum waiting time is 1365.0 minutes


It will be necessary to put the number of minutes spent at the airports into ranges to help with analysis. For example, 0 - 60 minutes will be termed as 'short', 61 - 180 minutes as long and so on. 

In [13]:
#Creating the ranges for the wait times and storing it in a column called wait_type
bins = [0, 60, 180, np.inf] # The ranges in which the times will be put
label_names = ['short', 'medium', 'long'] #The names the wait types will be called by
airlines['wait_type'] = pd.cut(airlines['wait_min'], bins = bins, labels = label_names)

#Verifying the changes
airlines['wait_type'][:3]

0    medium
1    medium
2    medium
Name: wait_type, dtype: category
Categories (3, object): ['short' < 'medium' < 'long']

The dataset being used here has been updated with the names of the first 200 passengers and provided it in a file called extras.py. The names will be loaded from this file and stored in a column in the dataframe.

In [14]:
from extras import names

In [15]:
#Getting the first 200 rowsfrom the dataframe
airlines_200 = airlines.loc[0:199].copy()
#Inserting the names as a column in position 2 in the dataframe
airlines_200.insert(loc = 2, column = 'full_name', value = names)
airlines_200.head(3)

,Unnamed: 0,id,full_name,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction,wait_type
0,0,1351,Melodie Stuart,Tuesday,UNITED INTL,KANSAI,asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied,medium
1,1,373,Dominic Shannon,Friday,ALASKA,SAN JOSE DEL CABO,canada/mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied,medium
2,2,2820,Quintessa Tillman,Thursday,DELTA,LOS ANGELES,west us,Hub,Gates 40-48,2018-12-31,70.0,Average,Somewhat safe,Neutral,medium


In [16]:
#Dropping the unneccessary column named Unnamed:0 
airlines_200.drop('Unnamed: 0', axis = 1, inplace = True)
airlines_200.head(3)

,id,full_name,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction,wait_type
0,1351,Melodie Stuart,Tuesday,UNITED INTL,KANSAI,asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied,medium
1,373,Dominic Shannon,Friday,ALASKA,SAN JOSE DEL CABO,canada/mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied,medium
2,2820,Quintessa Tillman,Thursday,DELTA,LOS ANGELES,west us,Hub,Gates 40-48,2018-12-31,70.0,Average,Somewhat safe,Neutral,medium


The client wants a seprate columns for first name and last name. In order to do that, we need to drop all the honorary titles. The titles include 'Dr.', 'Mr.', 'Miss', 'Ms.', 'Mrs.'

In [17]:
#Looking at the first 10 full_names
airlines_200['full_name'][:10]

0             Melodie Stuart
1            Dominic Shannon
2          Quintessa Tillman
3    Dr. Christine Nicholson
4            Regina Clements
5            Colleen Harding
6            Kaitlin Cochran
7               Molly Norton
8               Richard Lott
9             Matthew Nguyen
Name: full_name, dtype: object

In [18]:
#Looking at the last 10 full_names
airlines_200['full_name'][-10:]

190      Mr. Eaton Vazquez
191          Silas Clemons
192            Quinn Barry
193            Orson Pratt
194        Constance Morse
195       Ms. Vanna Rivera
196        Miss Venus Lowe
197        Amethyst Nieves
198    Miss Vivian Foreman
199     Miss Wendy Griffin
Name: full_name, dtype: object

In [19]:
# Replace "Dr." with empty string ""
airlines_200['full_name'] = airlines_200['full_name'].str.replace("Dr.", "")

# Replace "Mr." with empty string ""
airlines_200['full_name'] = airlines_200['full_name'].str.replace("Mr.", "")

# Replace "Miss" with empty string ""
airlines_200['full_name'] = airlines_200['full_name'].str.replace("Miss", "")

# Replace "Ms." with empty string ""
airlines_200['full_name'] = airlines_200['full_name'].str.replace("Ms.", "")

# Assert that full_name has no honorifics
assert airlines_200['full_name'].str.contains('Ms.|Mr.|Miss|Dr.').any() == False

In [20]:
#Creating the first name and last name columns 
airlines_200['first_name'] = airlines_200.apply(lambda row: row['full_name'].strip().split(' ')[0], axis = 1)
airlines_200['last_name'] = airlines_200.apply(lambda row: row['full_name'].strip().split(' ')[1], axis = 1)
airlines_200.head(3)

,id,full_name,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction,wait_type,first_name,last_name
0,1351,Melodie Stuart,Tuesday,UNITED INTL,KANSAI,asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied,medium,Melodie,Stuart
1,373,Dominic Shannon,Friday,ALASKA,SAN JOSE DEL CABO,canada/mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied,medium,Dominic,Shannon
2,2820,Quintessa Tillman,Thursday,DELTA,LOS ANGELES,west us,Hub,Gates 40-48,2018-12-31,70.0,Average,Somewhat safe,Neutral,medium,Quintessa,Tillman


The weekdays can be grouped as either a weekday or weekend.

In [21]:
# Create mappings and replace
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}

airlines_200['day_week'] = airlines['day'].replace(mappings)

In [22]:
airlines_200.head(2)

,id,full_name,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction,wait_type,first_name,last_name,day_week
0,1351,Melodie Stuart,Tuesday,UNITED INTL,KANSAI,asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied,medium,Melodie,Stuart,weekday
1,373,Dominic Shannon,Friday,ALASKA,SAN JOSE DEL CABO,canada/mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied,medium,Dominic,Shannon,weekday


In [23]:
airlines_200.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   id             200 non-null    int64   
 1   full_name      200 non-null    object  
 2   day            200 non-null    object  
 3   airline        200 non-null    object  
 4   destination    200 non-null    object  
 5   dest_region    200 non-null    object  
 6   dest_size      200 non-null    object  
 7   boarding_area  200 non-null    object  
 8   dept_time      200 non-null    object  
 9   wait_min       200 non-null    float64 
 10  cleanliness    200 non-null    object  
 11  safety         200 non-null    object  
 12  satisfaction   200 non-null    object  
 13  wait_type      200 non-null    category
 14  first_name     200 non-null    object  
 15  last_name      200 non-null    object  
 16  day_week       200 non-null    object  
dtypes: category(1), float64(1), int64(1

In [24]:
del airlines_200['full_name']

In [25]:
first_name = airlines_200['first_name'].tolist()
last_name = airlines_200['last_name'].tolist()

airlines_200.drop(['first_name', 'last_name'], axis = 1, inplace = True)
airlines_200.head(2)

,id,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction,wait_type,day_week
0,1351,Tuesday,UNITED INTL,KANSAI,asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied,medium,weekday
1,373,Friday,ALASKA,SAN JOSE DEL CABO,canada/mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied,medium,weekday


In [26]:
airlines_200.insert(loc = 2, column = 'first_name', value = first_name)
airlines_200.insert(loc = 3, column = 'last_name', value = last_name)

In [27]:
airlines_200.head(3)

,id,day,first_name,last_name,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction,wait_type,day_week
0,1351,Tuesday,Melodie,Stuart,UNITED INTL,KANSAI,asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied,medium,weekday
1,373,Friday,Dominic,Shannon,ALASKA,SAN JOSE DEL CABO,canada/mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied,medium,weekday
2,2820,Thursday,Quintessa,Tillman,DELTA,LOS ANGELES,west us,Hub,Gates 40-48,2018-12-31,70.0,Average,Somewhat safe,Neutral,medium,weekday
